In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import cv2


In [ ]:
IMG_PATH = "../input/uw-madison-gi-tract-image-segmentation/train/"# folder are name as case

In [ ]:

pd.set_option('display.max_column', 90)
pd.set_option('display.max_row', 200)

In [ ]:
train = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
test = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
print(train.shape)
print(train.dtypes.value_counts())

<h1> Data Exploratory </h1>
<ul>
    <h2> Objectif:</h2>
    <br>
    <li><h3> understand our dataset</h3></li>
    <br><h2>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;If I understand what I need to do is: First Identify organs and Second put a binary mask on it   </h2><br>
    <h2>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;So maybe I can use some of no segmentation part as testing Image   </h2><br>
    <li><h3> thinking of basic model</h3></li>
</ul>
<br>
<br>
<h1> CheckList de base </h1>
<br>
<ul>
    <h2>Annalyse de Forme:</h2>
    <br>
    <li><h3> variable Target:</h3></li>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;---> From Class --> segment organ</h3>
    <li><h3> Nombre de ligne et de colonne:</h3></li>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;--->115488 rows × 3 columns ----> with NaN value</h3>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;--->columns created: case 	day 	slice 	filename 	imagepath 	tumor</h3>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;--->rows deleted: no segmented image</h3>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;--->33913 rows × 9 columns ----> after process value</h3><br>
    <li><h3> Type de Variable:</h3></li>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;--->object    3</h3>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;. . .dtype: int64</h3><br>
    <li><h3> Annalyse des donnée manquantes:</h3></li>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;--->segmentation    0.70635 % Missing Value</h3><br>
</ul>
<br>
<br>
<ul>
    <h2> Annalyse de fond:</h2><br>
    <li><h3> Visualisation de la Target</h3></li>
    <li><h3> Signification des variables:</h3></li>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;--->segmentation    ROI = organ position rle_encoded mask</h3>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;--->class    Contain 3 Body Region</h3>
    <ul>
        <li>---> large_bowel    0.415327 %</li>
        <li>---> small_bowel    0.330286 %</li>
        <li>---> stomach        0.254386 %</li>
        <li>---> * large_bowel    14085 segments</li>
        <li>---> * small_bowel    11201 segments</li>
        <li>---> * stomach         8627 segments</li>
    </ul>
    <h3>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;--->id    Contain Case evolution througth day</h3><br><br>
    <li><h3> Relation Variable / Target</h3></li>
    <li><h3> Annalyse des donnée manquantes</h3></li>
</ul>


*     segmentation    0.70635
*     id              0.00000
*     class           0.00000
  
*     0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    0.70635 ----> no organs segmented
*     1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    0.29365 ----> organ segmented



<ul>
    <li></li>
    <li></li>
    <li></li>
</ul>

In [ ]:
plt.figure(figsize=(25, 25))
sns.heatmap(train.isna(), cbar=False);

In [ ]:
(train.isna().sum() / train.shape[0]).sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(28, 8))
train['class'].value_counts(normalize=True).plot.pie();

In [ ]:

# DATASET GLOBAL TRANSFORMATION

item_box = []
for item in train['id']:
    items = str(item)
    new_item = items.replace('_d', ' d').replace('_s', ' s')
    clean = new_item.split()
    item_box.append(clean)
    

cases = []
days = []
slices = []
for item in item_box:
    cases.append(item[0])
    days.append(item[1])
    slices.append(item[2])

day_numbers = []
for item in days:
    items = str(item)
    new_item = items.replace('day', '')
    day_numbers.append(int(new_item))
    
slice_numbers = []
for item in slices:
    items = str(item)
    new_item = items.replace('slice_', '')
    slice_numbers.append(int(new_item))
    
case_numbers = []
for item in cases:
    items = str(item)
    new_item = items.replace('case', '')
    case_numbers.append(int(new_item))

# I need to encode class
stomach_class      = [1, 0, 0]
large_bowel_class  = [0, 1, 0]
small_bowel_class = [0, 0, 1]

stomach_class_n      = 0
large_bowel_class_n  = 1
small_bowel_class_n  = 2

encoded = []
numerical_encoded = []

for item in train['class']:
    if str(item) == 'stomach':
        encoded.append(stomach_class)
        numerical_encoded.append(stomach_class_n)
        
    elif str(item) == 'large_bowel':
        encoded.append(large_bowel_class)
        numerical_encoded.append(large_bowel_class_n)
        
    elif str(item) == 'small_bowel':
        encoded.append(small_bowel_class)
        numerical_encoded.append(small_bowel_class_n)
        
    else:
        print('somethings went wrong when encoding class')

        
        
        
# list to dataframes        
class_encode = pd.Series(encoded)
class_numerical_encode = pd.Series(numerical_encoded)
cases = pd.Series(cases)
case_int = pd.Series(case_numbers)
days = pd.Series(days)
day_number = pd.Series(day_numbers)
slicing = pd.Series(slices)
slice_number = pd.Series(slice_numbers)

train['classencoded'] = class_encode
train['classnumericalencoded'] = class_numerical_encode
train['case'] = cases
train['caseintegrer'] = case_int
train['day'] = days
train['daynumber'] = day_number
train['slice'] = slicing
train['slicenumber'] = slice_number

print(train.shape)
train

In [ ]:

for i, col in enumerate(train.select_dtypes('int')):
    plt.figure(figsize=(5, 5));
    sns.distplot(train[col]);

In [ ]:

plt.figure(figsize=(28, 8))
train['case'].value_counts(normalize=True).plot.pie()

#print(train['case'].value_counts(normalize=True).sort_values(ascending=False))

our Distribution of case seems to be homogenous

Now let's try to open up some images, and add ImagePaths to our dataframe

IMG_PATH = "../input/uw-madison-gi-tract-image-segmentation/train/"


    ---- train:
    ---------- case101
    ------------------ case101_day20
    -------------------------------- scans
    ------------------------------------- slice
    ------------------------------------- slice
    ------------------------------------- slice
    ------------------------------------- ...


In [ ]:

cases = train['case'].tolist()
days = train['day'].tolist()
slices = train['slice'].tolist()
scans = 'scans'

paths = []
files_name = []

for i, item in enumerate(cases):
    day = days[i]
    folder_path = f'{IMG_PATH}{item}/{item}_{day}/{scans}/'
    
    arr = os.listdir(folder_path)
    for file in arr:
        if str(file[:int(len(str(slices[i])))]) == str(slices[i]):
            img_path = f'{IMG_PATH}{item}/{item}_{day}/{scans}/{file}'
            files_name.append(file)
            paths.append(img_path)


files = pd.Series(files_name)
paths = pd.Series(paths)

train['filename'] = files
train['imagepath'] = paths


# here all work but need some optimization

del cases
del days
del slices
del scans
del paths
del files_name
del files

train

In [ ]:
# new thinking NaN mean no tumor ?    so if nan = 0 else = 1 ?      WRONG NaN mean NO SEGMENT
#train.dropna(axis=0, subset=['segmentation'], inplace=True)

In [ ]:
# create new column tumor  0 = NO Tumor          1 = !! Tumor
check = train['segmentation'].tolist()
tumors = []

for i, item in enumerate(check):
    if str(item) == 'NaN' or str(item) == 'nan':
        #print(0)
        tumors.append('0')
    else:
        #print(1)
        tumors.append('1')
        
        
tumors = pd.Series(tumors)
train['tumor'] = tumors

del tumors

train

In [ ]:
print(train['tumor'].value_counts(normalize=True).sort_values(ascending=False))

plt.figure(figsize=(28, 8))
train['tumor'].value_counts(normalize=True).plot.pie();

*  as we can see here we need some balance into our dataset because our proportion are 70% no tumor and 30% tumor ??
*  I will keep thinking like it for now and focusing on 30% of tumor for now and see what I can found here

In [ ]:
(train.isna().sum() / train.shape[0]).sort_values(ascending=False)

In [ ]:
tumor_data = train.copy()

tumor_data.dropna(axis=0,inplace=True)
tumor_data

In [ ]:
print(tumor_data['class'].value_counts(normalize=True).sort_values(ascending=False))

plt.figure(figsize=(28, 8))
tumor_data['class'].value_counts(normalize=True).plot.pie();

I think I have maybe to much data for large_bowel

In [ ]:
tumor = tumor_data['imagepath'].tolist()
ROI = tumor_data['segmentation'].tolist()
for i, item in enumerate(tumor[:1]):
    img = pltimg.imread(f'{item}')
    print(ROI[i])
    plt.imshow(img, cmap='gray')

In [ ]:
# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def mask2rle(img):
    """
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formatted
    """
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle2mask(mask_rle: str, label=1, shape=img.shape):
    """
    mask_rle: run-length as string formatted (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background

    """
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = label
    return img.reshape(shape)  # Needed to align to RLE direction

* Note that the image filenames include 4 numbers (ex. 276_276_1.63_1.63.png). These four numbers are slice height / width (integers in pixels) and heigh/width pixel spacing (floating points in mm). The first two defines the resolution of the slide. The last two record the physical size of each pixel.

* slice_       276_ 276_ 1.63_ 1.63.png
* slice_ 0065_ 266_ 266_ 1.50_ 1.50.png -----> slice height/width --> 266x266 px  px_size = 1.50x1.50 mm
* on google : 1px = 0.2645833333 mm
* slice_ 0065_ 266_ 266_ 1.50_ 1.50.png -----> slice height/width --> 266x266 px  px_size = 1.50x1.50 mm ---> 1.5 / 0.2645833333 = 5.669291339296919
* the training annotations are provided as RLE-encoded masks, and the images are in 16-bit grayscale PNG format.

In [ ]:
ratio = 1.5 / 0.2645833333
print(266 * ratio)

'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'crest', 'crest_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'flare', 'flare_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'mako', 'mako_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket', 'rocket_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'vlag', 'vlag_r', 'winter', 'winter_r'

so the next thing to do is to put large_bowel, small_bowel and stomach on the same image


In [ ]:
#https://www.kaggle.com/zocainviken/organs-annalyze

stomach_rle_masks = tumor_data['segmentation'].tolist()
stomach_img_path = tumor_data['imagepath'].tolist()
stomach_slices = tumor_data['slice'].tolist()
stomach_slices = stomach_slices[:30]
stomach_rle_masks = stomach_rle_masks[:30]
stomach_img_path = stomach_img_path[:30]

columns = 3

plt.figure(figsize=(50, 50))

for i, mask in enumerate(stomach_rle_masks):
    
    plt.subplot(int(len(stomach_rle_masks) / columns + 1), columns, i + 1)
    
    color_img = cv2.imread(f'{stomach_img_path[i]}', 1)
    gray = pltimg.imread(f'{stomach_img_path[i]}')

    # Now create a mask and create its inverse mask also
    mask = rle2mask(str(mask), label=1, shape=gray.shape)
    mask_inv = cv2.bitwise_not(mask)
    
    foreground = cv2.bitwise_and(gray,gray,mask = mask)

    # Now black-out the area 
    color_background = cv2.bitwise_and(color_img,color_img,mask = mask_inv)

    # Put it in and modify the main image
    color_img[mask >= 1] = [0,100,0]
    plt.title(f'{stomach_slices[i]}')
    plt.imshow(color_img)

In [ ]:
print(tumor_data['class'].value_counts())
tumor_data['class'].value_counts(normalize=True).plot.pie();

In [ ]:
tumor_data.shape

In [ ]:
train